# Data Preprocessing

# Dataset 

The original data set provided by Rossmann are available in: **Dataset:** https://www.kaggle.com/c/rossmann-store-sales/data

The dataset published by the Rossmann has two parts: the first part is train.csv which comprises about 2.5 years of daily sales data for 1115 different Rossmann stores, resulting in a total number of 1017210 records; the second part is store.csv which describes additional details about each of these 1115 stores.  

In addition to the original data, external data was also used:
* weather
* store_states: mapping of store to the German state they are in
* state: list of German state names

The complete set of all data can be downloaded from: http://files.fast.ai/part2/lesson14/rossmann.tgz


## Dataset Description
The dataset provide historical sales data for **1,115** Rossmann stores.   

**Data Fields**

* **Id** - an Id that represents a (Store, Date) duple within the test set
* **Store** - a unique Id for each store
* **Sales** - the turnover for any given day (this is what you are predicting)
* **Customers** - the number of customers on a given day
* **Open** - an indicator for whether the store was open: 0 = closed, 1 = open
* **StateHoliday** - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
* **SchoolHoliday** - indicates if the (Store, Date) was affected by the closure of public schools
* **StoreType** - differentiates between 4 different store models: a, b, c, d
* **Assortment** - describes an assortment level: a = basic, b = extra, c = extended
* **CompetitionDistance** - distance in meters to the nearest competitor store
* **CompetitionOpenSince[Month/Year]** - gives the approximate year and month of the time the nearest competitor was opened
* **Promo** - indicates whether a store is running a promo on that day
* **Promo2** - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
* **Promo2Since[Year/Week]** - describes the year and calendar week when the store started participating in Promo2
* **PromoInterval** - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store.

## Goal
To forecast the **_Sales_** column for the test set. 

The training period data ranges from 2013-01-01 to 2015-07-31.  
The test period ranges from 2015-08-01 to 2015-09-17, so the task is to predict sales for 48 days.

### Evaluation Metric
Evaluation Metric: RMSPE (Root Mean Squared Percent Error).  
Basically means we are intrested in the ratio of the correct answer and predicted answer.  
Difference between the logs is the same as the ratio.

## Files
* **train.csv** - historical data (2013-01-01 to 2015-07-31) including Sales. 
* **test.csv** - historical data (2015-08-01 to 2015-09-17) excluding Sales
* **sample_submission.csv** - a sample submission file in the correct format
* **store.csv** - supplemental information about the stores
* **store_states** - maping of store to the German state thet are in
* **state** - list of German state names

## Summary

### 1. Create Dataset
* Merge different relevant csv files to create a single data frame 

### 2. Feature Engineering - Categoricals
* Replace binary categories as booleans (Easy for NN to handle)
* Create additional columns which may be useful for prediction.
* Address Missing values

### 3. Feature Engineering - Time Series
* Running averages
* Time until next event
* Time since last event  

### 4. Save as feather format

In [6]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

## Load Libraries

In [23]:
from fastai.imports import *
from fastai.structured import *
from fastai.column_data import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics
import altair as alt
from altair import Chart, X, Y, Axis, SortField
alt.renderers.enable('notebook')
from pandas import Timestamp
#import ggplot
np.set_printoptions(threshold=50, edgeitems=20)
from pdpbox import pdp
from plotnine import *

/Users/karthik/Desktop/code/fastai/old/Rossmann_Store_Sales


# Create Dataset
* Merge different relevant csv files to create a single data frame

## File Location

In [28]:
path_raw = 'data/raw/'
!ls {path_raw}

sample_submission.csv store_states.csv      weather.csv
state_names.csv       test.csv
store.csv             train.csv


## Load Data 
Load all the csv files as pandas dataframe into a list of **_tables_**

In [29]:
table_names = ['train','store','store_states','state_names','weather','test']
tables = [pd.read_csv(f'{path_raw }{fname}.csv', low_memory=False) for fname in table_names]
#Check if all csv files have been loaded
len(tables)

6

## Preliminary View of the Data

In [30]:
for t in tables:
    display (t.head(3))

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"


,Store,State
0,1,HE
1,2,TH
2,3,NW


,StateName,State
0,BadenWuerttemberg,BW
1,Bayern,BY
2,Berlin,BE


,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0


In [31]:
train, store, store_states, state_names, weather, test = tables

**Number of rows in train and test set**

In [32]:
len(train), len(test)

(1017209, 41088)

### Replace binary categories as booleans 
* Convert _State Holidays_ to booleans

In [33]:
#train['StateHoliday']!='0'
train['StateHoliday'] = train['StateHoliday']!='0'
test['StateHoliday'] = test['StateHoliday']!='0'

### Merge .csv files to create a single data frame
* Datframes are merged using a **_left outer join_**.  
* In outer join, everyrtime a value in the joining field on the left table does not have a corresponding value on the right table, the corresponding row in the new table has **_Null_** values for all right tables.
* Check the merge operation by checking for the null values post-join.

In [34]:
# Merge dataframes using Python merge function.
# Merging opertaion performed based on outer join.
# If two dataframes have the same column names, then the common coloum for the merged dataframe is appended with a suffix.
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

### Combine _Weather_ and _State_ 

In [35]:
#weather.head() 
#state_names.head()
print ("Before join:",len(weather), len(state_names))
weather = join_df(weather,state_names, "file","StateName")
print ("After join:",len(weather))

Before join: 15840 16
After join: 15840


### Combine _Store_ and _State_ 

In [36]:
print ("Before join:",len(store), len(state_names))
store = join_df(store, store_states, "Store")
print ("After join:",len(store))
#store.head(1)


Before join: 1115 16
After join: 1115


### Combine _Train_ & _Test_ with _Store_

In [37]:
print ("Train Set - Before join:",len(train), len(store))
print ("Train Set - Before join:",len(train), len(store))
joined = join_df(train, store, "Store")
joined_test = join_df(test,store,"Store")
print ("Test Set - After join:",len(train))
print ("Test Set - After join:",len(test))
print ("Check for Null values",len(joined[joined.StoreType.isnull()]), len(joined_test[joined_test.StoreType.isnull()]))

Train Set - Before join: 1017209 1115
Train Set - Before join: 1017209 1115
Test Set - After join: 1017209
Test Set - After join: 41088
Check for Null values 0 0


### Expand the date-time into additional fields (Feature Engineering)
The add_datepart() method generates date fields from a complete datetime for the purpose of constructing categoricals.

**add_datepart():** 
* Converts a date column to many columns containing the information from the date. 
  
**Parameters:**
* df: A pandas data frame. 
* fldname: A string that is the name of the date column we wish to expand.
    If it is not already a datetime64 series, it will be converted to one with pd.to_datetime.
* drop: If true then the original date column will be removed.

In [38]:
#weather.head(2)
#Expanding date field into additional fields
print ("Before date expand - weather columns:", weather.keys())
add_datepart(weather, "Date", drop=False)
print ("Before date expand - weather columns:", weather.keys())
add_datepart(train, "Date", drop=False)
add_datepart(test, "Date", drop=False)

Before date expand - weather columns: Index(['file', 'Date', 'Max_TemperatureC', 'Mean_TemperatureC',
       'Min_TemperatureC', 'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC',
       'Max_Humidity', 'Mean_Humidity', 'Min_Humidity',
       'Max_Sea_Level_PressurehPa', 'Mean_Sea_Level_PressurehPa',
       'Min_Sea_Level_PressurehPa', 'Max_VisibilityKm', 'Mean_VisibilityKm',
       'Min_VisibilitykM', 'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h',
       'Max_Gust_SpeedKm_h', 'Precipitationmm', 'CloudCover', 'Events',
       'WindDirDegrees', 'StateName', 'State'],
      dtype='object')
Before date expand - weather columns: Index(['file', 'Date', 'Max_TemperatureC', 'Mean_TemperatureC',
       'Min_TemperatureC', 'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC',
       'Max_Humidity', 'Mean_Humidity', 'Min_Humidity',
       'Max_Sea_Level_PressurehPa', 'Mean_Sea_Level_PressurehPa',
       'Min_Sea_Level_PressurehPa', 'Max_VisibilityKm', 'Mean_VisibilityKm',
       'Min_VisibilitykM', 'Max_

### Combine _Weather_ and _Joined_ based on 'State' and 'Date' columns


In [39]:
joined['Date'] = pd.to_datetime(joined['Date'])
joined_test['Date'] = pd.to_datetime(joined_test['Date'])
joined = join_df(joined, weather, ["State","Date"])
joined_test = join_df(joined_test, weather, ["State","Date"])
print ("Check for Null values",len(joined[joined.Mean_TemperatureC.isnull()]),len(joined_test[joined_test.Mean_TemperatureC.isnull()]))


Check for Null values 0 0


### Delete duplicate columns 
* Delete duplicate columns(i.e) the columns with suffix '_y'

In [40]:
# Delete duplicate columns
print("Before deleting duplicate columns",joined.keys())
for df in (joined, joined_test):
    for c in df.columns:
        if c.endswith('_y'):
            if c in df.columns: df.drop(c, inplace=True, axis=1)
print("After deleting duplicate columns",joined.keys())




Before deleting duplicate columns Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'State', 'file', 'Max_TemperatureC',
       'Mean_TemperatureC', 'Min_TemperatureC', 'Dew_PointC', 'MeanDew_PointC',
       'Min_DewpointC', 'Max_Humidity', 'Mean_Humidity', 'Min_Humidity',
       'Max_Sea_Level_PressurehPa', 'Mean_Sea_Level_PressurehPa',
       'Min_Sea_Level_PressurehPa', 'Max_VisibilityKm', 'Mean_VisibilityKm',
       'Min_VisibilitykM', 'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h',
       'Max_Gust_SpeedKm_h', 'Precipitationmm', 'CloudCover', 'Events',
       'WindDirDegrees', 'StateName', 'Year', 'Month', 'Week', 'Day',
       'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
       'Is_quarter_end', 'Is_quarter_start'

In [41]:
joined.shape

(1017209, 56)

##  Feature Engineering - Categoricals


* Address missing values
* Create additional columns which may be useful for prediction.


### Address Missing/Null Values

In [42]:
print ('CompetitionOpenSinceYear(Null values):',len(joined[joined.CompetitionOpenSinceYear.isnull()]))
print ('CompetitionOpenSinceMonth(Null values):',len(joined[joined.CompetitionOpenSinceMonth.isnull()]))
print ('Promo2SinceYear(Null values):',len(joined[joined.Promo2SinceYear.isnull()]))
print ('Promo2SinceWeek(Null values):',len(joined[joined.Promo2SinceWeek.isnull()]))

CompetitionOpenSinceYear(Null values): 323348
CompetitionOpenSinceMonth(Null values): 323348
Promo2SinceYear(Null values): 508031
Promo2SinceWeek(Null values): 508031


We fill the **missing values** for the year with an arbitary signal value that doesn't appear in the data.
* **_CompetitionOpenSinceYear (Null values)_** replaced with 1900
* **_CompetitionOpenSinceMonth (Null values)_** reaplced with 1
* **_Promo2SinceYear (Null values) replaced_** with 1900
* **_Promo2SinceWeek (Null values) replaced_** with 1

In [43]:
for df in (joined,joined_test):
    df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
    df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
    df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)

In [44]:
print ('CompetitionOpenSinceYear(Null values):',len(joined[joined.CompetitionOpenSinceYear.isnull()]))
print ('CompetitionOpenSinceMonth(Null values):',len(joined[joined.CompetitionOpenSinceMonth.isnull()]))
print ('Promo2SinceYear(Null values):',len(joined[joined.Promo2SinceYear.isnull()]))
print ('Promo2SinceWeek(Null values):',len(joined[joined.Promo2SinceWeek.isnull()]))

CompetitionOpenSinceYear(Null values): 0
CompetitionOpenSinceMonth(Null values): 0
Promo2SinceYear(Null values): 0
Promo2SinceWeek(Null values): 0


### Add additional columns
* CompetitionDaysOpen
    * Remove any erroneous values/outliers
* CompetitionMonthsOpen
    * Limit the maximum to 2 years to limit the number of unique categories
* Promo2Days
    * Remove any erroneous values/outliers
* Promo2Weeks
    * Limit the maximum to 25 to limit the number of unique categories

### Add columns - _CompetitionDaysOpen_ & _CompetitionMonthsOpen_

In [45]:
print("Date.(head(2)",joined['Date'].head(2))

for df in (joined,joined_test):
    df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear, 
                                                     month=df.CompetitionOpenSinceMonth, day=15))
    df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days

print ("CompetitionOpenSince.head(2)",joined["CompetitionOpenSince"].head(2))
print ("CompetitionDaysOpen.head(2)",joined["CompetitionDaysOpen"].head(2))

Date.(head(2) 0   2015-07-31
1   2015-07-31
Name: Date, dtype: datetime64[ns]
CompetitionOpenSince.head(2) 0   2008-09-15
1   2007-11-15
Name: CompetitionOpenSince, dtype: datetime64[ns]
CompetitionDaysOpen.head(2) 0    2510
1    2815
Name: CompetitionDaysOpen, dtype: int64


### Replace erroneous data and outliers

In [46]:
for df in (joined,joined_test):
    df.loc[df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
    df.loc[df.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0

### Convert CompetitionMonthsOpen variable to a categorical variable
* CompetitionMonthsOpen variable is truncated to be no more than 24 months - to use it as a categorical variable.
* Categorical variables, due to embeddings, have more flexibility in how the neural net can use them.

In [47]:
for df in (joined,joined_test):
    df["CompetitionMonthsOpen"] = df["CompetitionDaysOpen"]//30
    df.loc[df.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24
joined.CompetitionMonthsOpen.unique()

array([24,  3, 19,  9,  0, 16, 17,  7, 15, 22, 11, 13,  2, 23, 12,  4, 10,  1, 14, 20,  8, 18,  6, 21,  5])

### Add columns - Promo2Since & Promo2Days

In [48]:
def create_promo2since(x):
    return Week(x.Promo2SinceYear,x.Promo2SinceWeek).monday()

#df.apply(): runs a piece of Python code over every row in a dataframe   
for df in (joined,joined_test):
    df["Promo2Since"] = pd.to_datetime(df.apply(create_promo2since, axis=1).astype(pd.datetime))
    df["Promo2Days"] = df.Date.subtract(df["Promo2Since"]).dt.days

### Convert Promo2Weeks variable to a categorical variable
* Promo2Weeks variable is truncated to be no more than 25 weeks - to use it as a categorical variable.
* Categorical variables, due to embeddings, have more flexibility in how the neural net can use them.

In [49]:
for df in (joined,joined_test):
    df.loc[df.Promo2Days<0, "Promo2Days"] = 0
    df.loc[df.Promo2SinceYear<1990, "Promo2Days"] = 0
    df["Promo2Weeks"] = df["Promo2Days"]//7
    df.loc[df.Promo2Weeks<0, "Promo2Weeks"] = 0
    df.loc[df.Promo2Weeks>25, "Promo2Weeks"] = 25
    df.Promo2Weeks.unique()

## Feature Engineering - Time Series
**When working with time series data it is important to explain relationships across rows as opposed to columns.**

For our dataset, we have a table where:
* For each store for each date - the store may/may not have a promo going on that date.
* There might be a school holiday in that region of that store that date.
* There might be a state holiday in that region for that store that date.

The above can be classified as **_events_**.  

Often a time series will show something happening before and after the event.     
For example:  If there’s a holiday coming up, the sales may be higher/lower before/after the holiday.  

We create two new columns for three events: 
* School Holiday, 
* State Holiday, 
* Promo

We also create new columns for:
* How long is it going to be until the next time this event happens.
* How long has it been since the last time that event happened.
* Running averages
* Time until next event
* Time since last event

In [52]:
# Sort by store and date. 
# Loop through each store and keep track of the event (last_date).
# Append to the result the number of days since the last event.

def get_elapsed(fld, pre):
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    last_store = 0
    res = []

# Iterating through a DataFrame and extracting specific fields out of a row is time consuming. 
# Its faster to iterate through a numpy array.
# convert dataframes to numpy array using (.values) and zip it.

    for s,v,d in zip(df.Store.values,df[fld].values, df.Date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1))
    df[pre+fld] = res

In [53]:
columns = ["Date", "Store", "Promo", "StateHoliday", "SchoolHoliday"]

In [54]:
df = train[columns].append(test[columns])
df.head()

,Date,Store,Promo,StateHoliday,SchoolHoliday
0,2015-07-31,1,1,False,1
1,2015-07-31,2,1,False,1
2,2015-07-31,3,1,False,1
3,2015-07-31,4,1,False,1
4,2015-07-31,5,1,False,1


In [55]:
fld = 'SchoolHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')

In [56]:
# Sort values in ascending order
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')
df.head()

,Date,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday
0,2015-09-17,1,1,False,0,13.0,NaN
856,2015-09-16,1,1,False,0,12.0,NaN
1712,2015-09-15,1,1,False,0,11.0,NaN
2568,2015-09-14,1,1,False,0,10.0,NaN
3424,2015-09-13,1,0,False,0,9.0,NaN


In [57]:
fld = 'StateHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')
df.head()

,Date,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday
0,2015-09-17,1,1,False,0,13.0,NaN,105.0,NaN
856,2015-09-16,1,1,False,0,12.0,NaN,104.0,NaN
1712,2015-09-15,1,1,False,0,11.0,NaN,103.0,NaN
2568,2015-09-14,1,1,False,0,10.0,NaN,102.0,NaN
3424,2015-09-13,1,0,False,0,9.0,NaN,101.0,NaN


In [58]:
fld = 'Promo'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')
df.head()

,Date,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo
0,2015-09-17,1,1,False,0,13.0,NaN,105.0,NaN,0.0,0.0
856,2015-09-16,1,1,False,0,12.0,NaN,104.0,NaN,0.0,0.0
1712,2015-09-15,1,1,False,0,11.0,NaN,103.0,NaN,0.0,0.0
2568,2015-09-14,1,1,False,0,10.0,NaN,102.0,NaN,0.0,0.0
3424,2015-09-13,1,0,False,0,9.0,NaN,101.0,NaN,9.0,-1.0


In [59]:
#Set active index to date
df = df.set_index("Date")

In [60]:
columns = ['SchoolHoliday', 'StateHoliday', 'Promo']
for o in ['Before', 'After']:
    for p in columns:
        a = o+p
        df[a] = df[a].fillna(0).astype(int)
df.head()

,Store,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo
Date,,,,,,,,,,
2015-09-17,1,1,False,0,13,0,105,0,0,0
2015-09-16,1,1,False,0,12,0,104,0,0,0
2015-09-15,1,1,False,0,11,0,103,0,0,0
2015-09-14,1,1,False,0,10,0,102,0,0,0
2015-09-13,1,0,False,0,9,0,101,0,9,-1


### Rolling Averages

For each store **_groupby(store)_**, we sort by date **_sort_index()_** and count the number of events of interest **_sum()_** defined in _columns_ for the following/previous week **_rolling(7)_**.

In [61]:
bwd = df[['Store']+columns].sort_index().groupby("Store").rolling(7, min_periods=1).sum()
fwd = df[['Store']+columns].sort_index(ascending=False).groupby("Store").rolling(7, min_periods=1).sum()


In [62]:
bwd.drop('Store',1,inplace=True)
bwd.reset_index(inplace=True)
fwd.drop('Store',1,inplace=True)
fwd.reset_index(inplace=True)
df.reset_index(inplace=True)

#### Merge rolling values into the dataframe

In [63]:
df = df.merge(bwd, 'left', ['Date', 'Store'], suffixes=['', '_bw'])
df = df.merge(fwd, 'left', ['Date', 'Store'], suffixes=['', '_fw'])

In [64]:
df.drop(columns,1,inplace=True)
df["Date"] = pd.to_datetime(df.Date)

In [65]:
df.columns

Index(['Date', 'Store', 'AfterSchoolHoliday', 'BeforeSchoolHoliday',
       'AfterStateHoliday', 'BeforeStateHoliday', 'AfterPromo', 'BeforePromo',
       'SchoolHoliday_bw', 'StateHoliday_bw', 'Promo_bw', 'SchoolHoliday_fw',
       'StateHoliday_fw', 'Promo_fw'],
      dtype='object')

In [66]:
joined = join_df(joined, df, ['Store', 'Date'])
joined_test = join_df(joined_test, df, ['Store', 'Date'])

In [67]:
#Check if you have better accuracy without this
joined = joined[joined.Sales!=0]

## Save as feather format
* The feature engineered datasets are saved in fearther format.  
* The feather format saves the csv files on the disk in the same same format as it is stored on the RAM.  
* This enables the data to loaded much faster as compared to loading a csv file or a pickeled file.

In [68]:
path_processed = 'data/processed/'
#Backup Data
joined.reset_index(inplace=True)
joined_test.reset_index(inplace=True)

joined.to_feather(f'{path_processed}joined')
joined_test.to_feather(f'{path_processed}joined_test')